In [1]:
# ts module is currently in alpha. updated with new releases.
# create env with python=3.6 (control if still valid)
# conda install -c anaconda bottleneck
# pip install pycaret-ts-alpha

In [2]:
# seems to handle univariate ts for the moment

In [3]:
import pandas as pd
import datetime
import numpy as np
import plotly.express as px

# from pycaret.datasets import get_data
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

In [4]:
# y = get_data('airline')
# y

In [5]:
# DATA
df = pd.read_csv('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/nyc_energy_consumption.csv')
df.head()

,timeStamp,demand,precip,temp
0,2012-01-01 00:00:00,4937.5,0.0,46.13
1,2012-01-01 01:00:00,4752.1,0.0,45.89
2,2012-01-01 02:00:00,4542.6,0.0,45.04
3,2012-01-01 03:00:00,4357.7,0.0,45.03
4,2012-01-01 04:00:00,4275.5,0.0,42.61


In [6]:
# Processing
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
# demand missing from 2017-08-10
df = df[df['timeStamp']<'2017-08-10']

# Resample
df = df.set_index('timeStamp')
df_daily = df[['demand']].resample('D').sum()
# df_daily = df_daily.reset_index()
df_daily.head()

,demand
timeStamp,
2012-01-01,118916.0
2012-01-02,127270.9
2012-01-03,146292.3
2012-01-04,152070.4
2012-01-05,147125.9


In [7]:
train = df_daily[df_daily.index<='2017-06-30']
test = df_daily[(df_daily.index>'2017-06-30')]
y_train = train['demand']
y_test = test['demand']
y_train

timeStamp
2012-01-01    118916.000
2012-01-02    127270.900
2012-01-03    146292.300
2012-01-04    152070.400
2012-01-05    147125.900
                 ...    
2017-06-26    163784.767
2017-06-27    158494.334
2017-06-28    154514.432
2017-06-29    166068.785
2017-06-30    181930.910
Freq: D, Name: demand, Length: 2008, dtype: float64

## Time Series Module (alpha)

In [8]:
# initialize setup
exp = TimeSeriesExperiment()
exp.setup(data=y_train, fh=40, fold=3, session_id=42)

,Description,Value
0,session_id,42
1,Original Data,"(2008, 1)"
2,Missing Values,False
3,Transformed Train Set,"(1968,)"
4,Transformed Test Set,"(40,)"
5,Fold Generator,ExpandingWindowSplitter
6,Fold Number,3
7,CPU Jobs,-1
8,Use GPU,False
9,Log Experiment,False


In [9]:
best_baseline_models = exp.compare_models(n_select=3)

,Model,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
arima,ARIMA,5935.9898,89472461.3396,0.0428,0.0440,0.3014,0.2033
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,5942.7469,94351776.4025,0.0431,0.0440,0.2684,0.3867
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,5974.0828,95927694.3873,0.0432,0.0444,0.2046,1.6767
snaive,Seasonal Naive Forecaster,6249.6362,95480508.3321,0.0453,0.0465,0.1957,0.0133
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,6311.4238,98756571.7755,0.0460,0.0470,0.1911,2.1333
ets,ETS,6428.9683,105112267.5923,0.0466,0.0479,0.0760,0.1600
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,6468.2989,98613615.1066,0.0475,0.0483,0.1902,2.0733
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,6515.7068,104402126.6218,0.0471,0.0485,0.1085,0.6067
exp_smooth,Exponential Smoothing,6516.9326,104455289.9784,0.0473,0.0486,0.0608,1.7500
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,7569.1175,116128557.5947,0.0573,0.0567,-0.0076,0.3833


In [10]:
best_baseline_models

[ARIMA(maxiter=50, method='lbfgs', order=(1, 0, 0), out_of_sample_size=0,
       scoring='mse', scoring_args=None, seasonal_order=(0, 1, 0, 7),
       start_params=None, suppress_warnings=False, trend=None,
       with_intercept=True),
 BaseCdsDt(degree=1, deseasonal_model='additive',
           regressor=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                   colsample_bytree=1.0, importance_type='split',
                                   learning_rate=0.1, max_depth=-1,
                                   min_child_samples=20, min_child_weight=0.001,
                                   min_split_gain=0.0, n_estimators=100,
                                   n_jobs=-1, num_leaves=31, objective=None,
                                   random_state=42, reg_alpha=0.0,
                                   reg_lambda=0.0, silent=True, subsample=1.0,
                                   subsample_for_bin=200000, subsample_freq=0),
           sp=1, window_length=

In [11]:
best_tuned_models = [exp.tune_model(model) for model in best_baseline_models]

,cutoff,MAE,RMSE,MAPE,SMAPE,R2
0,2017-01-21,4785.1522,36944893.8002,0.0361,0.0358,0.4452
1,2017-03-02,6285.1754,69021080.5272,0.0462,0.0480,-0.0990
2,2017-04-11,7408.5643,184972869.9922,0.0517,0.0539,0.2178
Mean,nan,6159.6306,96979614.7732,0.0447,0.0459,0.1880
SD,nan,1074.6763,63583698.9685,0.0064,0.0075,0.2232


In [12]:
mean_blender = exp.blend_models(best_tuned_models, method = 'mean')

,cutoff,MAE,RMSE,MAPE,SMAPE,R2
0,2017-01-21,4903.2710,34957277.3807,0.0368,0.0367,0.4751
1,2017-03-02,5969.8417,63085892.0798,0.0438,0.0454,-0.0045
2,2017-04-11,7400.8430,182972987.1945,0.0517,0.0539,0.2263
Mean,nan,6091.3186,93672052.2183,0.0441,0.0453,0.2323
SD,nan,1023.2412,64180980.9935,0.0061,0.0070,0.1958


In [13]:
final_model = exp.finalize_model(mean_blender)

In [14]:
pred = exp.predict_model(final_model)
rmse_valid = np.sqrt(np.mean((pred[0]-y_test)**2))
mape_valid = np.mean(np.abs((pred[0]-y_test)/y_test))
print('RMSE_valid', rmse_valid)
print('mape_valid', mape_valid)

,Model,MAE,RMSE,MAPE,SMAPE,R2
0,EnsembleForecaster,nan,nan,nan,nan,nan


RMSE_valid 19409.160706283645
mape_valid 0.08496630016233545
